In [1]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModel
from tools.standard_name_utils import optimize_parameters
from tools.common_utils import highlight_diff, read_jsonl

model_path = "./models/chatglm3-6b-01"
top_p=0.7 
temperature=0.95

/home/reymond/anaconda3/envs/afac2024/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = AutoModel.from_pretrained(model_path, trust_remote_code=True).half().cuda()
model = model.eval()

Setting eos_token is not supported, use the default one.
Setting pad_token is not supported, use the default one.
Setting unk_token is not supported, use the default one.
Loading checkpoint shards: 100%|██████████| 3/3 [00:57<00:00, 19.19s/it]


In [3]:
response, history = model.chat(tokenizer, "你好", history=[], top_p=top_p, temperature=temperature)
print(response)

你好👋！我是人工智能助手 ChatGLM3-6B，很高兴见到你，欢迎问我任何问题。


In [4]:
json_file_path = 'data/dev_api.json'
data_list = read_jsonl(json_file_path)

df = pd.DataFrame(data_list)
df.head()

,conversations
0,"[{'role': 'user', 'content': '你现在是一个金融领域专家，你需要..."
1,"[{'role': 'user', 'content': '你现在是一个金融领域专家，你需要..."
2,"[{'role': 'user', 'content': '你现在是一个金融领域专家，你需要..."
3,"[{'role': 'user', 'content': '你现在是一个金融领域专家，你需要..."
4,"[{'role': 'user', 'content': '你现在是一个金融领域专家，你需要..."


In [5]:
from random import random

total_eval_count = 0.0
correct_count = 0.0

for index, row in df.iterrows():
    if random() <= 0.9:
        continue
    
    single_user_row = row["conversations"][0]
    single_assis_row = row["conversations"][1]
    
    input = single_user_row["content"]
    output = single_assis_row["content"]
    
    
    response, history = model.chat(tokenizer=tokenizer, query=input, history=[], top_p=top_p, temperature=temperature)
    
    # optimized_resp = optimize_parameters(response, fund_standard_name, stock_standard_name)
    
    total_eval_count += 1
    if response == output:
        correct_count += 1
    else:
        print("-----data index-----")
        print(index)
        print("-----query input-----")
        print(input)
        print("-----output diff-----")
        print(highlight_diff(output, response))
        print(response)
        print()
    
    if total_eval_count == 20:
        break
    
print("预测正确的比例：" + f"{correct_count / total_eval_count :.2%}")

-----data index-----
13
-----query input-----
你现在是一个金融领域专家，你需要通过编排api来得到用户query的答案，输出json格式。query是：请问世茂今天的开盘价和成交量是多少，以及主力资金的净流入情况如何？ 
 query中使用到的api可能是：股票查询_查询主力资金净流入、条件选股_查询主力资金净流入、股票查询_查询主力资金流入、条件选股_查询主力资金流入、股票查询_查询主力资金流出、条件选股_查询主力资金流出、股票查询_查询成交量、股票查询_查询开盘价、条件选股_查询成交量、条件选股_查询开盘价、条件选股_查询每股经营性现资金流、股票查询_查询收盘价、股票查询_查询成交额、股票查询_查询总流入、条件选股_查询收盘价、条件选股_查询成交额、条件选股_查询总流入、股票查询_查询投资收入、条件选股_查询投资收入、基金查询_查询单位净值、股票查询_查询净融资性现金流、股票查询_查询净经营性现金流、股票查询_查询净投资性现金流、条件选股_查询净融资性现金流、条件选股_查询净经营性现金流、条件选股_查询净投资性现金流、股票查询_查询每股经营性现金流、基金查询_查询单位净值同步日期、基金查询_查询近5年买入持有1年历史盈利概率、基金查询_查询近3年买入持有1年历史盈利概率、基金查询_查询近一年买入持有6个月历史盈利概率、基金查询_查询规模、基金查询_查询行业、基金查询_查询类型、基金查询_查询板块、基金查询_查询代码、股票查询_查询跌停价、股票查询_查询涨停价、股票查询_查询最高价、股票查询_查询最低价、股票查询_查询总资产、股票查询_查询总流出、股票查询_查询当前价、股票查询_查询净利率、股票查询_查询净利润、条件选股_查询跌停价、条件选股_查询涨停价、条件选股_查询最高价、条件选股_查询最低价、条件选股_查询总资产、数值计算_加法计算、数值计算_减法计算、数值计算_乘法计算、数值计算_除法计算、逻辑计算_与计算、逻辑计算_或计算
-----output diff-----
{"relevant APIs": [{"api_id": "0", "api_name": "查询代码", "rely_apis": [], "tool_name": "股票查询"}, {"api_id": "1", "api_name": "